In [ ]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false
import sys
import os
import time
import logging
import datetime
import json
from datetime import datetime, timedelta
import yaml
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import requests
import pandas_gbq
from dreams_core.googlecloud import GoogleCloud as dgc
from dreams_core import core as dc
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
import progressbar


# load dotenv
load_dotenv()


# import local files if necessary
# pyright: reportMissingImports=false
sys.path.append('..//src')
import training_data as td
importlib.reload(td)
import feature_engineering as fe
importlib.reload(fe)
import coin_wallet_metrics as cwm
importlib.reload(cwm)
import modeling as m
importlib.reload(m)
import insights as i
importlib.reload(i)
import utils as u
importlib.reload(u)

# load configs
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')

# configure logger
logger = dc.setup_logger()
logger.setLevel(logging.INFO)

# Custom format function for displaying numbers
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
importlib.reload(u)
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')

## Base Tables

In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)

config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')

# retreive market data
market_data_df = td.retrieve_market_data()
market_data_df,_ = td.fill_market_data_gaps(market_data_df,config['data_cleaning']['max_gap_days'])
market_data_df,_,_ = cwm.split_dataframe_by_coverage(
    market_data_df,
    start_date=config['training_data']['training_period_start'],
    end_date=config['training_data']['modeling_period_end'],
    id_column='coin_id'
)
prices_df = market_data_df[['coin_id','date','price']].copy()


# retrieve transfers data
transfers_df = td.retrieve_transfers_data(
    config['training_data']['training_period_start'],
    config['training_data']['modeling_period_start'],
    config['training_data']['modeling_period_end']
    )


# compile profits_df
profits_df = td.prepare_profits_data(transfers_df, prices_df)
profits_df = td.calculate_wallet_profitability(profits_df)
profits_df,_ = td.clean_profits_df(profits_df, config['data_cleaning'])

## Experiment Setup

In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
importlib.reload(u)
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')

# initial steps for this model
filtered_market_data_df = market_data_df[market_data_df['coin_id'].isin(profits_df['coin_id'])]

training_data_tuples = []
training_data_dfs = []

### Prices Metrics

In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')

# dataset variables
dataset_category = 'time_series'
dataset_name = 'market_data'
dataset_df = filtered_market_data_df.copy()
# declare dataset configs based on variables
dataset_config = config['datasets'][dataset_category][dataset_name]
dataset_metrics_config = metrics_config[dataset_category][dataset_name]
# calculate metrics for each value column
value_column = 'price'
# a value_column-specific df will be used for feature generation
value_column_config = config['datasets'][dataset_category][dataset_name][value_column]
value_column_metrics_config = metrics_config[dataset_category][dataset_name][value_column]
value_column_df = dataset_df[['date','coin_id',value_column]].copy()

# check if there are any time series indicators to add, e.g. sma, ema, etc
if 'indicators' in value_column_metrics_config:
    # add the indicators as new columns in the value_column_df
    value_column_metrics_df, _ = cwm.generate_time_series_indicators(
        value_column_df,
        config,
        value_column_metrics_config['indicators'],
        value_column,
        id_column='coin_id'
    )

else:
    # if no indicators are needed, pass through coins in the original df that have values for all dates
    value_column_metrics_df, _, _ = cwm.split_dataframe_by_coverage(
        value_column_df,
        config['training_data']['training_period_start'],
        config['training_data']['training_period_end'],
        id_column='coin_id'
    )

value_column_metrics_df.head()


<!-- ## Prices Features -->

In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')

# dataset variables
dataset_category = 'time_series'
dataset_name = 'market_data'
dataset_df = filtered_market_data_df.copy()

# declare dataset configs based on variables
dataset_config = config['datasets'][dataset_category][dataset_name]
dataset_metrics_config = metrics_config[dataset_category][dataset_name]



# calculate metrics for each value column
for value_column in list(dataset_metrics_config.keys()):

    # a value_column-specific df will be used for feature generation
    value_column_config = config['datasets'][dataset_category][dataset_name][value_column]
    value_column_metrics_config = metrics_config[dataset_category][dataset_name][value_column]
    value_column_df = dataset_df[['date','coin_id',value_column]].copy()

    # check if there are any time series indicators to add, e.g. sma, ema, etc
    if 'indicators' in value_column_metrics_config:

        # calculate and merge all indicators in the config
        all_indicator_dfs = []
        for indicator in value_column_metrics_config.keys():
            indicator_df, _ = cwm.generate_time_series_indicators(
                value_column_df,
                config,
                value_column_metrics_config['indicators'],
                value_column,
                id_column='coin_id'
            )
            all_indicator_dfs.append(indicator_df)

        # join all generated indicators for the value_column together
        value_column_indicators_df = all_indicator_dfs[0]
        for indicator_df in all_indicator_dfs[1:]:
            value_column_indicators_df = pd.merge(value_column_indicators_df, indicator_df, on='date', how='outer')

    else:
        # if no indicators are needed, pass through coins in the original df that have values for all dates
        value_column_metrics_df, _, _ = cwm.split_dataframe_by_coverage(
            value_column_df,
            config['training_data']['training_period_start'],
            config['training_data']['training_period_end'],
            id_column='coin_id'
        )

    # generate features from the metrics
    value_column_features_df, value_column_tuple = fe.convert_dataset_metrics_to_features(
        value_column_metrics_df,
        value_column_config,
        dataset_metrics_config,
        config,
        modeling_config
    )

    logger.info('Generated features for %s.%s.%s',
                dataset_category, dataset_name, value_column)

    training_data_tuples.append(value_column_tuple)
    training_data_dfs.append(value_column_features_df)


### Wallet Cohorts

In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')
logger.setLevel(logging.INFO)

# dataset variables
dataset_category = 'wallet_cohorts'


for cohort_name in metrics_config[dataset_category]:

    # load configs
    dataset_metrics_config = metrics_config[dataset_category][cohort_name]
    dataset_config = config['datasets'][dataset_category][cohort_name]
    cohort_description = dataset_config['description']

    # identify wallets in the cohort
    cohort_summary_df = cwm.classify_wallet_cohort(profits_df, dataset_config)
    cohort_wallets = cohort_summary_df[cohort_summary_df['in_cohort']==True]['wallet_address']

    # If no cohort members were identified, continue
    if len(cohort_wallets) == 0:
        logger.info("No wallets identified as members of cohort '%s'", cohort_name)
        continue

    # generate cohort buysell_metrics
    cohort_metrics_df = cwm.generate_buysell_metrics_df(profits_df,config['training_data']['training_period_end'],cohort_wallets)

    # generate features from the metrics
    dataset_features_df, dataset_tuple = fe.convert_dataset_metrics_to_features(
        cohort_metrics_df,
        dataset_config,
        dataset_metrics_config,
        config,
        modeling_config
    )

    logger.info('Generated features for %s.%s',
                dataset_category, cohort_name)

    training_data_tuples.append(dataset_tuple)
    training_data_dfs.append(dataset_features_df)



In [ ]:
training_data_tuples = [
    ('price_2024-09-26_00-37_model_period_2024-09-01_preprocessed.csv', 'drop_records'),
    ('volume_2024-09-26_00-37_model_period_2024-09-01_preprocessed.csv', 'drop_records'),
    ('market_cap_2024-09-26_00-37_model_period_2024-09-01_preprocessed.csv', 'drop_records'),
    ('cohort_10k_2024-09-26_00-48_model_period_2024-09-01_preprocessed.csv', 'fill_zeros'),
    ('cohort_50k_2024-09-26_00-48_model_period_2024-09-01_preprocessed.csv', 'fill_zeros'),
    ('cohort_whales_2024-09-26_00-49_model_period_2024-09-01_preprocessed.csv', 'fill_zeros'),
    ('cohort_normies_2024-09-26_00-50_model_period_2024-09-01_preprocessed.csv', 'fill_zeros')
]

### Metadata

In [ ]:
dataset_category = 'coin_facts'
dataset_name = 'coin_metadata'


# load configs
dataset_config = config['datasets'][dataset_category][dataset_name]


# generate features
metadata_df = td.retrieve_metadata_data()
metadata_features_df = td.generate_coin_metadata_features(metadata_df, config)
metadata_features_df.head()

# save flattened output
flattened_output_directory = os.path.join(modeling_config['modeling']['modeling_folder'],'outputs/flattened_outputs')
flattened_metadata_df, flattened_metadata_filepath = fe.save_flattened_outputs(
    metadata_features_df,
    flattened_output_directory,
    dataset_config['description'],
    config['training_data']['modeling_period_start']
)

# check preprocessed file
preprocessed_metadata_df, preprocessed_metadata_output_path = fe.preprocess_coin_df(
    flattened_metadata_filepath,
    modeling_config,
    dataset_config
)

metadata_tuple = (preprocessed_metadata_output_path.split('preprocessed_outputs/')[1], dataset_config['fill_method'])


training_data_tuples.append(metadata_tuple)
training_data_dfs.append(preprocessed_metadata_df)

### Construct Model

In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')
logger.setLevel(logging.INFO)


# merge training data
modeling_folder = modeling_config['modeling']['modeling_folder']
training_data_df, merge_logs_df = fe.create_training_data_df(modeling_folder, training_data_tuples)

# create the target variable df
target_variable_df,_ = fe.create_target_variables_mooncrater(filtered_market_data_df, config['training_data'], modeling_config)

# merge the two into the final model input df
model_input_df = fe.prepare_model_input_df(training_data_df, target_variable_df, modeling_config['modeling']['target_column'])

# split the df into train and test sets
X_train, X_test, y_train, y_test = m.split_model_input(
    model_input_df,
    modeling_config['modeling']['target_column'],
    modeling_config['modeling']['train_test_split'],
    modeling_config['modeling']['random_state']
)

# 3.4 Train the model using the current configuration and log the results
modeling_folder = modeling_config['modeling']['modeling_folder']
model, model_id = m.train_model(X_train, y_train, modeling_folder, modeling_config['modeling']['model_params'])

# 3.5 Evaluate the model's performance on the test set
metrics = m.evaluate_model(model, X_test, y_test, model_id, modeling_folder)

# 3.6 Log the experiment results for this configuration
m.log_trial_results(modeling_folder, model_id)

metrics

In [ ]:
# Assuming `model` is your trained model and `feature_names` is a list of your feature names
feature_importances = model.feature_importances_
feature_names = X_train.columns  # Replace with the correct source of your feature names if different

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importances
})

# Sort by importance (optional)
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Display the feature importance
feature_importance_df.sort_values('importance',ascending=False)
# feature_importance_df.sort_values('feature',ascending=False)

## tests failing

In [ ]:
max_gap_days = 2

# Input data
prices_df_gaps_below_max = pd.DataFrame({
    'coin_id': ['coin1', 'coin1', 'coin1', 'coin2', 'coin2'],
    'date': pd.to_datetime(['2024-01-01', '2024-01-03', '2024-01-04',
                            '2024-01-01', '2024-01-03']),
    'price': [100, 102, 103, 200, 202]
})

# Expected data after filling
expected_data = {
    'date': pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04',  # coin1
                            '2024-01-01', '2024-01-02', '2024-01-03']),  # coin2
    'coin_id': ['coin1', 'coin1', 'coin1', 'coin1',
                'coin2', 'coin2', 'coin2'],
    'price': [100.0, 100, 102, 103,
                200, 200, 202]
}
expected_df = pd.DataFrame(expected_data)[['date', 'coin_id', 'price']]

# Run the function
prices_filled_df, outcomes_df = td.fill_market_data_gaps(prices_df_gaps_below_max, max_gap_days)

# Reorder columns for comparison
prices_filled_df = prices_filled_df[['date', 'coin_id', 'price']]

# Assertions
pd.testing.assert_frame_equal(prices_filled_df, expected_df)
assert all(outcomes_df['outcome'] == 'gaps below threshold')

In [ ]:
np.array_equal(prices_filled_df.values,expected_df.values)


In [ ]:
prices_filled_df

In [ ]:
expected_df

In [ ]:
prices_df_gaps_below_max

In [ ]:
prices_filled_df